In [1]:
# Auto Reload
%load_ext autoreload
%autoreload 2

In [22]:
# Import Modules
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import json
from pprint import pprint
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from math import log
import re


# NLTK Modules
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

# Import Custom Modules


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wesle\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wesle\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wesle\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# Run Scraping Program
# !python src/web_scraper.py

# Intake Scraped Data

In [10]:
# Get Data
data = pd.read_csv('data/data.csv')
data.head(3)

,Unnamed: 0,song,artist,featured,rank,year,lyrics,lyrics_state,song_id,lyrics_owner_id,primary_artist_url
0,0,See You Again,Wiz Khalifa,NaN,1,2015,"It's been a long day without you, my friend\nA...",True,720401,341761,https://genius.com/artists/Wiz-khalifa
1,1,Trap Queen,Fetty Wap,NaN,2,2015,"RGF productions\nRemy Boyz, yah-ah\n1738, ayy\...",True,496445,104344,https://genius.com/artists/Fetty-wap
2,2,Watch Me,Silento,NaN,3,2015,"Whip, nae nae\nWhip, whip, nae nae\nWhip, nae ...",True,1743010,1696010,https://genius.com/artists/Silento


### Lyric String Example
> With the intake lyric data, it seems that there needs to be a couple things cleaned. Casing, punctuation, and new-lines

In [43]:
sample = data['lyrics'][0]
sample[:len(sample)//5]

"It's been a long day without you, my friend And I'll tell you all about it when I see you again We've come a long way from where we began Oh, I'll tell you all about it when I see you again When I see you again  Damn, who knew? All the planes we flew, good things we been through That I'd be standing right here talking to you 'Bout another path, I know we loved to hit the road and laugh But something told me that it wouldn't last Had to switch"

### Text Cleaning

In [40]:
# Clean New-line breaks
data['lyrics'] = data['lyrics'].apply(lambda x: " ".join(x.split('\n')))

In [44]:
sample = data['lyrics'][0]
sample[:len(sample)//5]

"It's been a long day without you, my friend And I'll tell you all about it when I see you again We've come a long way from where we began Oh, I'll tell you all about it when I see you again When I see you again  Damn, who knew? All the planes we flew, good things we been through That I'd be standing right here talking to you 'Bout another path, I know we loved to hit the road and laugh But something told me that it wouldn't last Had to switch"

In [48]:
# Create Text Cleaning Function
def clean_text(df, text_field, new_text_field_name):
    df_cop = df.copy()
    df_cop[new_text_field_name] = df_cop[text_field].str.lower()
    df_cop[new_text_field_name] = df_cop[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    # remove numbers
    df_cop[new_text_field_name] = df_cop[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    
    return df_cop

data = clean_text(data, 'lyrics', 'lyrics')

In [50]:
sample = data['lyrics'][0]
sample[:len(sample)//5]

'its been a long day without you my friend and ill tell you all about it when i see you again weve come a long way from where we began oh ill tell you all about it when i see you again when i see you again  damn who knew all the planes we flew good things we been through that id be standing right here talking to you bout another path i know we loved to hit the road and laugh but something told me that it wouldnt last had to s'